In [1]:
import numpy as np

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Jupiter
from poliastro.ephem import Ephem
from poliastro.frames import Planes
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver
from poliastro.plotting import StaticOrbitPlotter
from poliastro.util import norm, time_range
from poliastro import iod
from poliastro.plotting import OrbitPlotter3D
from poliastro.threebody.flybys import compute_flyby

import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
## Main dates (1 набор дат)
date_launch = Time("2029-05-28", scale="utc").tdb
date_flyby_Earth = Time("2031-02-14", scale="utc").tdb
date_arrival = Time("2033-11-27", scale="utc").tdb

In [15]:
earth_1 = Ephem.from_body(Earth, time_range(date_launch, end=date_launch))
earth_2 = Ephem.from_body(Earth, time_range(date_launch, end=date_flyby_Earth))
jup = Ephem.from_body(Jupiter, time_range(date_launch, end=date_arrival))

In [16]:
ss_earth_1 = Orbit.from_ephem(Sun, earth_1, date_launch)
ss_earth_2 = Orbit.from_ephem(Sun, earth_2, date_flyby_Earth)
ss_jup = Orbit.from_ephem(Sun, jup, date_arrival)

In [17]:
man_lambert_1 = Maneuver.lambert(ss_earth_1, ss_earth_2)
ss_trans_1, ss_target_1 = ss_earth_1.apply_maneuver(man_lambert_1, intermediate=True)

man_lambert_2 = Maneuver.lambert(ss_earth_2, ss_jup)
ss_trans_2, ss_target_2 = ss_earth_2.apply_maneuver(man_lambert_2, intermediate=True)

In [18]:
#1
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")


plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=310 * u.deg), color="blue", label="Transfer orbit E-E"
)

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=184 * u.deg), color="purple", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [19]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_1, date_launch, label="Earth")
plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth")

plotter.plot_trajectory(
    ss_trans_1.sample(max_anomaly=310 * u.deg), color="blue", label="Transfer orbit E-E"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [20]:
plotter = OrbitPlotter3D()
plotter.set_attractor(Sun)

plotter.plot_ephem(earth_2, date_flyby_Earth, label="Earth")
plotter.plot_ephem(jup, date_arrival, label="Jupiter")

plotter.plot_trajectory(
    ss_trans_2.sample(max_anomaly=184 * u.deg), color="purple", label="Transfer orbit E-J"
)

plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [9]:
r_earth, v_earth = Ephem.from_body(Earth, date_launch).rv(date_launch)
r_earth_2, v_earth_2 = Ephem.from_body(Earth, date_flyby_Earth).rv(date_flyby_Earth)
r_jup, v_jup = Ephem.from_body(Jupiter, date_arrival).rv(date_arrival)

In [10]:
# Фазовые координаты Земли_1

print (f"{r_earth [0]}")
print (f"{r_earth [1]}")
print (f"{r_earth [2]}"'\n')

print (v_earth.to(u.km / u.s))

-60143098.70230435 km
-127695278.3750023 km
-55346793.51878729 km

[ 26.83043818 -10.96231978  -4.7531655 ] km / s


In [11]:
# Фазовые координаты Земли

print (f"{r_earth_2 [0]}")
print (f"{r_earth_2 [1]}")
print (f"{r_earth_2 [2]}"'\n')

print (v_earth_2.to(u.km / u.s))

-120654929.48869102 km
78590609.19266155 km
34078780.41113805 km

[-17.73708488 -22.38556682  -9.70433443] km / s


In [12]:
# Фазовые координаты Юпитера

print (f"{r_jup [0]}")
print (f"{r_jup [1]}")
print (f"{r_jup [2]}"'\n')

print (v_jup.to(u.km / u.s))

702163550.476614 km
-225350609.49954826 km
-113677735.79351984 km

[ 4.24896779 11.90523476  4.99939469] km / s


In [13]:
nh_tof_1 = date_flyby_Earth - date_launch # время полета Земля1 - Земля

(v0_Ea1, vk_earth), = iod.lambert(Sun.k, r_earth, r_earth_2, nh_tof_1)
nh_tof_1

<TimeDelta object: scale='tdb' format='jd' value=627.0000000013098>

In [14]:
# Гиперболический избыток скорости при отлете от Земли

V_inf = (norm(v0_Ea1 - v_earth)).to(u.km / u.s)
V_inf

<Quantity 9.52273852 km / s>

In [15]:
# Гиперболический избыток скорости при отлете от Земли

V_inf_vec = (v0_Ea1 - v_earth).to(u.km / u.s)
V_inf_vec

<Quantity [-0.03633689, -8.73664281, -3.78844309] km / s>

In [16]:
# Гиперболический избыток скорости при подлете к Земле_2

V_inf_end = (norm(vk_earth - v_earth_2)).to(u.km / u.s)
V_inf_end

<Quantity 9.53405902 km / s>

In [17]:
# Гиперболический избыток скорости при подлете к Земле_2

V_inf_end_vec = (vk_earth - v_earth_2).to(u.km / u.s)
V_inf_end_vec

<Quantity [ 4.9620877 , -7.47017201, -3.2361238 ] km / s>

In [23]:
nh_tof_2 = date_arrival - date_flyby_Earth # время полета Марс - Земля2

(v0_earth, vk_Jup), = iod.lambert(Sun.k, r_earth_2, r_jup, nh_tof_2)
nh_tof_2

<TimeDelta object: scale='tdb' format='jd' value=1016.999999975351>

In [24]:
# Гиперболический избыток скорости при отлете от Земли_2

V_inf_end_vec_0 = (norm(v0_earth - v_earth_2)).to(u.km / u.s)
V_inf_end_vec_0

<Quantity 9.50279013 km / s>

In [25]:
# Гиперболический избыток скорости при отлете от Марса

V_inf_end_vec_0_vec = (v0_earth - v_earth_2).to(u.km / u.s)
V_inf_end_vec_0_vec

<Quantity [-1.75272993, -9.28839219, -0.97812495] km / s>

In [26]:
# Разница подлетного и отлетного гиперболического избытка скорости

V_gip_diff = V_inf_end - V_inf_end_vec_0
V_gip_diff

<Quantity 0.03126889 km / s>

In [27]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_end_2 = (norm(vk_Jup - v_jup)).to(u.km / u.s)
V_inf_end_2

<Quantity 5.99571314 km / s>

In [28]:
# Гиперболический избыток скорости при подлете к Юпитеру

V_inf_end_2_vec = (vk_Jup - v_jup).to(u.km / u.s)
V_inf_end_2_vec

<Quantity [-2.49419997, -4.84434744, -2.50196732] km / s>

In [29]:
time_adv = nh_tof_1 + nh_tof_2
time_adv

<TimeDelta object: scale='tdb' format='jd' value=1643.9999999766608>

In [30]:
scalar_EM = r_earth[0]*r_earth_2[0] + r_earth[1]*r_earth_2[1] +r_earth[2]*r_earth_2[2] 
scalar_MJ = r_earth_2[0]*r_jup[0] + r_earth_2[1]*r_jup[1] +r_earth_2[2]*r_jup[2]

In [31]:
fi_EM = (scalar_EM/norm(r_earth)/norm(r_earth_2))
fi_MJ = (scalar_MJ/norm(r_earth_2)/norm(r_jup))

In [33]:
np.degrees(np.arccos(fi_EM))

<Quantity 102.00227656 deg>

In [35]:
np.degrees(np.arccos(fi_MJ))

<Quantity 164.32779665 deg>

In [36]:
## Необходимый импульс на опорной орбите 
from math import*

mu = 398600.4415 ## Гравитационный параметр Земли 
r = 6371.
R = r + 200.
V_gip = 9.5227385 ## Гиперболический избыток скорости при отлете от Земли

dV = sqrt (2 * mu / R + V_gip**2 ) - sqrt(mu/r)
dV * u.km / u.s

<Quantity 6.65055233 km / s>

In [37]:
## Необходимый импульс для торможения на эллиптическую орбиту Юпитера 
## Не уверен что это нужно

muJ = 126686534
r_p = 670900
r_a = 4000000
V_inf_jup = 5.9957131

print (sqrt ((2 * mu / R + V_inf_jup**2 )))
print (sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p))))

dV_el = sqrt ((2 * mu / R + V_inf_jup**2 )) - sqrt ((2*muJ*r_a)/(r_p*(r_a+r_p)))
dV_el * u.km / u.s

12.540720301096894
17.983782441612995


<Quantity -5.44306214 km / s>

In [38]:
## параметры участка Землия-Марс 

from math import*

to_rad = pi/180

ist_an_1 = 53.8207123 * to_rad
ist_an_2 = 311.8642125 * to_rad

exs_EM = (norm(r_earth_2)/norm(r_earth) - 1) / (cos(ist_an_1) - (norm(r_earth_2)/norm(r_earth) * cos(ist_an_2)))
p_EM = norm(r_earth)*(1+cos(ist_an_1)*exs_EM)
a_EM = p_EM/(1 - exs_EM*exs_EM)

print (f"Эксцентриситет {exs_EM}")
print (f"Фокальный параметр {p_EM}")
print (f"Большая полуось {a_EM}")

Эксцентриситет 0.39365319438005886
Фокальный параметр 186844842.59714365 km
Большая полуось 221108432.71359769 km


In [39]:
## параметры участка Марс-Юпитер 

ist_an_1_MJ = 347.5988856 * to_rad
ist_an_2_MJ = 183.1605174 * to_rad

exs_MJ = (norm(r_jup)/norm(r_earth_2) - 1) / (cos(ist_an_1_MJ) - (norm(r_jup)/norm(r_earth_2) * cos(ist_an_2_MJ)))
p_MJ = norm(r_earth_2)*(1+cos(ist_an_1_MJ)*exs_MJ)
a_MJ = p_MJ/(1 - exs_MJ*exs_MJ)

print (f"Эксцентриситет {exs_MJ}")
print (f"Фокальный параметр {p_MJ}")
print (f"Большая полуось {a_MJ}")

Эксцентриситет 0.6724631680250577
Фокальный параметр 245154642.5632055 km
Большая полуось 447531300.748567 km
